### Exploratory Analysis of Covid-19 Cases, Hospitalizations, and Hospital Capacity

#### Aaron McAdie

I've been really curious about how the current case load in the US compares with regional hospital beds, and which regions are approaching their capacity.  The folks at the New York times just open sourced their [county-level Covid-19 case dataset](https://github.com/nytimes/covid-19-data), and it provides a great jumping off point to probe the question.  It's also a good opportunity to brush up on Python, since I do ~90% of my day job using R.

Also, since I started working on this, I came across the [incredible modeling and data viz](https://covid19.healthdata.org/projections) done by the UW IHME team.  They have done a way better job than I have the subject matter expertise to match, so I'll pull in their projections and explore it from a few different angles.

In [1]:
import requests
import re
import zipfile
from bs4 import BeautifulSoup
import pandas as pd
import altair as alt

#### Dataset 1 - NYTimes Cases, thanks NYT!

In [2]:
cases = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')

In [3]:
cases.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


In [4]:
cases.dtypes

date       object
county     object
state      object
fips      float64
cases       int64
deaths      int64
dtype: object

In [5]:
cases = cases.assign(
    date = pd.to_datetime(cases['date']),
    fips = cases['fips'].astype(pd.Int32Dtype())
)

Cases are cumulative, we want new cases each day to estimate the hospital case load

In [6]:
cases['cases_shifted'] = (
    cases.groupby(['county', 'state'])
    .cases
    .shift(1)
    .fillna(0)
    .astype(int)
)

In [7]:
cases['cases_new'] = cases['cases'] - cases['cases_shifted']

Check logic by plotting King County data.  I've been wanting to see more presentations of new cases per day rather than the cumulative plots that are more commonly displayed to get a better feel for how we are 'flattening the curve'.

In [8]:
king = cases.query('county == "King" & state == "Washington"')

In [9]:
base = alt.Chart(king[['date', 'cases', 'cases_new']]).encode(alt.X('monthdate(date):O', title = 'Date'))
bars = base.mark_bar(color = '#65799b').encode(y = 'cases_new', tooltip = 'cases_new')
bars.properties(title = 'King County WA Epidemiological Curve (new cases per day)')

alt.Chart(...)

In [10]:
line = base.mark_line(color = '#e23e57').encode(y = 'cases', tooltip = 'cases')

(bars + line).properties(title = 'King County Cumulative and Daily Covid-19 Cases')

alt.LayerChart(...)

Passes sanity check.

In [11]:
statewide = (
    cases.groupby(['date', 'state'])
    .aggregate({'cases':'sum'})
    .reset_index()
)

In [12]:
statewide.head()

,date,state,cases
0,2020-01-21,Washington,1
1,2020-01-22,Washington,1
2,2020-01-23,Washington,1
3,2020-01-24,Illinois,1
4,2020-01-24,Washington,1


#### State by State Cases Over Time

In [13]:
(alt.Chart(statewide)
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q')
 .properties(title = 'Cumulative US Covid-19 Cases by Day', width = 600)
)

alt.Chart(...)

In [14]:
(alt.Chart(statewide[statewide['state'] != 'New York'])
 .mark_rect()
 .encode(x = alt.X('date:O', axis = alt.Axis(labels = False)),
         y = 'state:N',
         color = 'cases:Q',
         tooltip = ['date', 'state', 'cases'])
 .properties(title = 'Cumulative US Covid-19 Cases by Day (NY removed)', width = 600)
)

alt.Chart(...)

### Dataset 2 - Population and Demographics by County (U.S. Census Bureau)

In [15]:
pop = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2017/counties/asrh/cc-est2017-alldata.csv',
                 encoding='ISO-8859-1')

In [16]:
pop.head()

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
0,50,1,1,Alabama,Autauga County,1,0,54571,26569,28002,...,607,538,57,48,26,32,9,11,19,10
1,50,1,1,Alabama,Autauga County,1,1,3579,1866,1713,...,77,56,9,5,4,1,0,0,2,1
2,50,1,1,Alabama,Autauga County,1,2,3991,2001,1990,...,64,66,2,3,2,7,2,3,2,0
3,50,1,1,Alabama,Autauga County,1,3,4290,2171,2119,...,51,57,13,7,5,5,2,1,1,1
4,50,1,1,Alabama,Autauga County,1,4,4290,2213,2077,...,48,44,7,5,0,2,2,1,3,1


#### Join cases and census data to get per-capita infections

In [17]:
# Filter to get most recent census data (2017)
# strip the ' County' from the census data to join to cases
# start with total age

pop17 = (pop
       .query('YEAR == 10 and AGEGRP == 0')
       .assign(county_key = lambda x: x['CTYNAME'].str.extract(r'([A-Za-z\s\.\-\']*)(?=\sCounty|\sParish)'))
       .loc[:,['STNAME', 'CTYNAME', 'county_key', 'TOT_POP']]
      )

In [18]:
# Manually fix a few County/district names so they join to the NYT cases dataset

# All cases for the five boroughs of New York City 
# (New York, Kings, Queens, Bronx and Richmond counties) 
# are assigned to a single area called New York City.
nyc_idx = (pop17.CTYNAME.str.contains(r'New York|Kings|Queens|Bronx|Richmond')) \
    & (pop17.STNAME == 'New York') # and vs & is so confusing to me
    
pop17.loc[nyc_idx, 'county_key'] = 'New York City'

# D.C
pop17.loc[pop17.CTYNAME == 'District of Columbia', 'county_key'] = 'District of Columbia'

# make case insensitive
pop17['county_key'] = pop17.county_key.str.lower()
cases['county_key'] = cases.county.str.lower()

# in NYT data - "Cities like St. Louis and Baltimore that are administered separately from an adjacent 
# county of the same name are counted separately."
# strip off the -city and combine with the surrounding county to join correctly to the county-level census data
def rm_city(county_name):
    if re.search(r'(?<!new york)\scity', county_name):
        return re.sub(r'([A-Za-z\s\.\-\']*)(\scity)', r'\1', county_name)
    else:
        return(county_name)
    
cases['county_key'] = cases.county_key.apply(rm_city)

# aggregate across new key with combined cities and counties
cases = (cases
         .groupby(['county_key', 'county', 'state', 'date'])
         .aggregate({'cases': 'sum', 'deaths': 'sum', 'cases_shifted': 'sum', 'cases_new': 'sum'})
         .reset_index()
        )

In [19]:
pop17.loc[nyc_idx,]

,STNAME,CTYNAME,county_key,TOT_POP
347871,New York,Bronx County,new york city,1471160
351861,New York,Kings County,new york city,2648771
353191,New York,New York County,new york city,1664727
355091,New York,Queens County,new york city,2358582
355471,New York,Richmond County,new york city,479458


In [20]:
pop17_clean = (pop17
               .groupby(['county_key', 'STNAME'])
               .agg({'TOT_POP':'sum'})
               .reset_index()
              )

In [21]:
cases_percap = cases.merge(pop17_clean, how = 'left', left_on = ['county_key', 'state'], 
                           right_on = ['county_key', 'STNAME'])

In [22]:
cases_percap.head()

,county_key,county,state,date,cases,deaths,cases_shifted,cases_new,STNAME,TOT_POP
0,abbeville,Abbeville,South Carolina,2020-03-19,1,0,0,1,South Carolina,24722.0
1,abbeville,Abbeville,South Carolina,2020-03-20,1,0,1,0,South Carolina,24722.0
2,abbeville,Abbeville,South Carolina,2020-03-21,1,0,1,0,South Carolina,24722.0
3,abbeville,Abbeville,South Carolina,2020-03-22,1,0,1,0,South Carolina,24722.0
4,abbeville,Abbeville,South Carolina,2020-03-23,1,0,1,0,South Carolina,24722.0


In [23]:
# check mismatches
sum(pd.isnull(cases_percap.TOT_POP))

1625

In [24]:
cases_percap.loc[pd.isnull(cases_percap.TOT_POP), 'county_key'].value_counts()

unknown                              677
virginia beach                        36
alexandria                            33
james                                 33
carson                                33
anchorage                             33
harrisonburg                          32
fairbanks north star borough          29
williamsburg                          28
charlottesville                       28
ketchikan gateway borough             28
kenai peninsula borough               27
charles                               26
portsmouth                            25
norfolk                               25
doña ana                              25
newport news                          25
kansas                                25
suffolk                               24
matanuska-susitna borough             22
danville                              22
juneau and borough                    22
chesapeake                            21
manassas                              20
fredericksburg  

Still some mismatches to iron out, but mostly good

In [25]:
cases_percap['cases_per10k'] = cases_percap.cases / cases_percap.TOT_POP * 10000
cases_percap['cases_new_per1k'] = cases_percap.cases_new / cases_percap.TOT_POP * 1000
cases_percap['county_label'] = cases_percap.county + ', ' + cases_percap.STNAME

In [26]:
cases_percap.head()

,county_key,county,state,date,cases,deaths,cases_shifted,cases_new,STNAME,TOT_POP,cases_per10k,cases_new_per1k,county_label
0,abbeville,Abbeville,South Carolina,2020-03-19,1,0,0,1,South Carolina,24722.0,0.404498,0.04045,"Abbeville, South Carolina"
1,abbeville,Abbeville,South Carolina,2020-03-20,1,0,1,0,South Carolina,24722.0,0.404498,0.00000,"Abbeville, South Carolina"
2,abbeville,Abbeville,South Carolina,2020-03-21,1,0,1,0,South Carolina,24722.0,0.404498,0.00000,"Abbeville, South Carolina"
3,abbeville,Abbeville,South Carolina,2020-03-22,1,0,1,0,South Carolina,24722.0,0.404498,0.00000,"Abbeville, South Carolina"
4,abbeville,Abbeville,South Carolina,2020-03-23,1,0,1,0,South Carolina,24722.0,0.404498,0.00000,"Abbeville, South Carolina"


In [27]:
# cumulative cases for counties with > 1 million people
# save data as JSON for better performance with Altair

cases_percap_url = 'cases_percap.json'

cases_percap[cases_percap.TOT_POP > 1000000].to_json(cases_percap_url, orient = 'records')

# can't figure out why the file url isn't working, can read it in just fine with open('cases_percap.json')
# these data are small enough for just saving the dataset with the chart though
cases_percap_big = cases_percap.loc[cases_percap.TOT_POP > 1000000, 
                                    ['date', 'cases', 'cases_per10k', 'county_label', 'cases_new_per1k', 'STNAME', 'TOT_POP']]

(alt.Chart(cases_percap_big)
    .mark_line(color = 'grey', opacity = 0.6)
    .encode(x = 'date:T',
            y = 'cases_per10k:Q',
            color = 'county_label:N',
            tooltip = ['county_label:N', 'cases:Q'])
    .properties(height = 400, 
                width = 700,
               title = 'Per Capita Covid-19 Cases, Counties > 1M')
    .interactive())

alt.Chart(...)

Getting somewhere, but need a way to visually distinguish. Maybe a panel of plots with cross-filtering?

In [28]:
(alt.Chart(cases_percap_big)
    .mark_point()
    .encode(x = 'TOT_POP:Q',
            y = 'STNAME:N',
            tooltip = 'county_label'))

alt.Chart(...)

In [29]:
# combine the dot plot and the cumulative case plot, with cross-filtering
brush = alt.selection(type = 'interval', resolve = 'global')
base = alt.Chart(cases_percap_big)

pop_points = (base.mark_point()
             .encode(x = alt.X('TOT_POP:Q', axis = alt.Axis(title = 'Population')),
                     y = alt.Y('STNAME:N', axis = alt.Axis(title = 'State')))
             .add_selection(brush)
             .properties(height = 400, 
                         width = 200,
                         title = 'Counties > 1M (brush to select)'))

percap_cases_line = (base.mark_line(color = 'grey', opacity = 0.6)
            .encode(x = alt.X('date:T', axis = alt.Axis(title = 'Date')),
                    y = alt.Y('cases_per10k:Q', axis = alt.Axis(title = 'Cases per 10,000 people')),
                    color = 'county_label:N',
                    tooltip = 'county_label:N')
            .properties(height = 400, 
                        width = 450,
                        title = 'Per Capita Covid-19 Cases, Counties > 1M')
            .transform_filter(brush))

(pop_points | percap_cases_line).save('percap_covid_big_counties.html')

pop_points | percap_cases_line

alt.HConcatChart(...)

Wow, Altair is rad!

Stealing idea that Nate Cohn had for new cases vs prevalence, updating with new data
tweaking by using a moving average of new cases per capita rather than % change in cumulative
cases, since we are well into the pandemic, the same absolute number of new cases will be smaller and
smaller as a percent of the cumulative total

In [30]:
# create column for rolling 5 day average of new cases
cases_percap_big = cases_percap_big.sort_values(['county_label', 'date'])
#cases_percap_big[~cases_percap_big['cases_new_per1k'].isna()]

cases_percap_big['cases_new_per1k_MA'] = (cases_percap_big
                                          .groupby('county_label')['cases_new_per1k']
                                          .transform(lambda x: x.rolling(5).mean()))

In [31]:
lines = (alt.Chart(cases_percap_big)
    .mark_line()
    .encode(x = 'cases_per10k:Q',
            y = 'cases_new_per1k_MA:Q',
            stroke = 'county_label:N',
            tooltip = 'county_label:N')
    .interactive()
    .properties(height = 400, width = 700))

# get latest record per county for label
cases_percap_big['latest'] = cases_percap_big.groupby('county_label')['date'].transform('max')
cases_percap_big_latest = cases_percap_big[cases_percap_big.date == cases_percap_big.latest]

labels = (alt.Chart(cases_percap_big_latest)
          .mark_point()
          .encode(x = 'cases_per10k:Q',
                  y = 'cases_new_per1k_MA:Q')
          .mark_text(align = 'left',
                    baseline = 'middle',
                    color = 'grey')
          .encode(text = 'county_label:N'))

lines + labels

alt.LayerChart(...)

In [32]:
#oh snap, do the side-by side panel, with the scatter being most recent new cases per 1k, and the x being population (or 
#maybe log population), and then the right plot the new cases vs cases per 10k

# add a max cumulative cases column for the scatter plot, could do with the Altair call, but hopefully this will get the
# selection behavior I'm looking for

max_cases = (cases_percap_big
             .groupby('county_label')
             .agg({'cases_per10k': 'max'})
             .reset_index()
             .rename(columns = {'cases_per10k': 'max_cases_per10k'})
            )

cases_percap_big = cases_percap_big.merge(max_cases, how = 'inner', on = 'county_label')


brush2 = alt.selection(type = 'interval', 
                       empty = 'all')

scatter_pop = (alt.Chart(cases_percap_big)
               .mark_point()
               .encode(x = alt.X('TOT_POP:Q', 
                                 axis = alt.Axis(title = 'County Population (log2)'),
                                 scale = alt.Scale(type = 'log', base = 2)),
                       y = alt.Y('max_cases_per10k:Q',
                                axis = alt.Axis(title = 'Total Cases per 10,000 (log2)'),
                                scale = alt.Scale(type = 'log', base = 2)))
               .add_selection(brush2)
               .properties(height = 400, 
                           width = 200,
                           title = ['brush to select population range', 
                                    'drag selection to compare similar size counties'])
              )

spread_chart = (alt.Chart(cases_percap_big)
                .mark_line()
                .encode(x = alt.X('cases_per10k:Q',
                                  axis = alt.Axis(title = 'Cumulative Cases per 10,000')),
                        y = alt.Y('cases_new_per1k_MA:Q',
                                  axis = alt.Axis(title = '5-day Average of New Cases per 1,000')),
                        color = 'county_label:N',
                        tooltip = 'county_label:N')
                .transform_filter(brush2)
                .properties(height = 400, 
                            width = 450,
                            title = 'Transmittance vs Prevalence, Counties > 1M')
                .interactive()
               )

scatter_pop | spread_chart

alt.HConcatChart(...)

Can't figure out how to add the labels on the spread chart but pretty happy with this!

The biggest things that jumped out are that Miami-Dade and Cook Illinois are the two big counties that could be the next major oubreaks.  The virus is relatively prevalent in those communities, and the rate of new cases is also relatively high on a per-capita basis.

Of the smaller counties (note that this is relative, all counties plotted had more than 1 million residents in 2017), by far the most prevalence/fasted spread is in Suffolk and Nassau counties, which makes sense since these are adjacent to NYC.  Middlesex MA and Phildelphia PA look to be on a trajectory of rapid spread.  Wayne MI (which includes Detroit) has had quite a dramatic decrease in new cases over the last few days (provided they are still testing at the same rate).

### Dataset 3 - Hospitalization Rates by Age Group (CDC)

The CDC published a table of hospitalization, ICU, and death rates by age group, based on ~2500 hospitalizations in the US in Feb/Mar.  This could be used to get a rough estimate of particular types of beds needed, namely pediatric, which is of high interest for us here at Seattle Children's.

In [33]:
# get html from CDC
cdc_html = requests.get('https://www.cdc.gov/mmwr/volumes/69/wr/mm6912e2.htm')
soup = BeautifulSoup(cdc_html.content)

In [34]:
# find table
tables = soup.find_all('table')
tables[0].find_all('tbody')

[<tbody>
 <tr>
 <td align="left" valign="top">0–19 (123)</td>
 <td align="right" valign="bottom">1.6–2.5</td>
 <td align="right" valign="bottom">0</td>
 <td align="right" valign="bottom">0</td>
 </tr>
 <tr>
 <td align="left" valign="top">20–44 (705)</td>
 <td align="right" valign="bottom">14.3–20.8</td>
 <td align="right" valign="bottom">2.0–4.2</td>
 <td align="right" valign="bottom">0.1–0.2</td>
 </tr>
 <tr>
 <td align="left" valign="top">45–54 (429)</td>
 <td align="right" valign="bottom">21.2–28.3</td>
 <td align="right" valign="bottom">5.4–10.4</td>
 <td align="right" valign="bottom">0.5–0.8</td>
 </tr>
 <tr>
 <td align="left" valign="top">55–64 (429)</td>
 <td align="right" valign="bottom">20.5–30.1</td>
 <td align="right" valign="bottom">4.7–11.2</td>
 <td align="right" valign="bottom">1.4–2.6</td>
 </tr>
 <tr>
 <td align="left" valign="top">65–74 (409)</td>
 <td align="right" valign="bottom">28.6–43.5</td>
 <td align="right" valign="bottom">8.1–18.8</td>
 <td align="right" vali

In [35]:
table = tables[0].find('tbody')

rows_dict = dict()

for idx, row in enumerate(table.find_all('tr')):
    temp_list = list()
    for col in row.find_all('td'):
        temp_list.append(col.string)
    rows_dict[idx] = temp_list

In [36]:
cdc_df = pd.DataFrame(rows_dict).T
cdc_df.columns = ['age_range', 'hosp_rate', 'icu_rate', 'death_rate']
cdc_df

,age_range,hosp_rate,icu_rate,death_rate
0,0–19 (123),1.6–2.5,0,0
1,20–44 (705),14.3–20.8,2.0–4.2,0.1–0.2
2,45–54 (429),21.2–28.3,5.4–10.4,0.5–0.8
3,55–64 (429),20.5–30.1,4.7–11.2,1.4–2.6
4,65–74 (409),28.6–43.5,8.1–18.8,2.7–4.9
5,75–84 (210),30.5–58.7,10.5–31.0,4.3–10.5
6,≥85 (144),31.3–70.3,6.3–29.0,10.4–27.3
7,"Total (2,449)",20.7–31.4,4.9–11.5,1.8–3.4


In [37]:
# clean up a bit, extract relevant info into usable columns
# self-challenge by doing this with vectorized pandas functions rather than standard regex

cdc_df = (cdc_df
          .assign(n = lambda x: x['age_range'].str.extract(r'\(([0-9,]{3,})\)'),
                  age_range = lambda x: x['age_range'].str.extract(r'(.*)\s\([0-9,]{3,}\)'),
                  hosp_lo = lambda x: x['hosp_rate'].str.extract(r'([0-9\.]{2,}).[0-9\.]{2,}'),
                  hosp_hi = lambda x: x['hosp_rate'].str.extract(r'[0-9\.]{2,}.([0-9\.]{2,})'))
         )

cdc_df

,age_range,hosp_rate,icu_rate,death_rate,n,hosp_lo,hosp_hi
0,0–19,1.6–2.5,0,0,123,1.6,2.5
1,20–44,14.3–20.8,2.0–4.2,0.1–0.2,705,14.3,20.8
2,45–54,21.2–28.3,5.4–10.4,0.5–0.8,429,21.2,28.3
3,55–64,20.5–30.1,4.7–11.2,1.4–2.6,429,20.5,30.1
4,65–74,28.6–43.5,8.1–18.8,2.7–4.9,409,28.6,43.5
5,75–84,30.5–58.7,10.5–31.0,4.3–10.5,210,30.5,58.7
6,≥85,31.3–70.3,6.3–29.0,10.4–27.3,144,31.3,70.3
7,Total,20.7–31.4,4.9–11.5,1.8–3.4,"2,449",20.7,31.4


In [38]:
# strip off total into separate df
cdc_total = cdc_df[cdc_df.age_range.str.contains('Total')]
cdc_df = cdc_df[~cdc_df.age_range.str.contains('Total')]

### Dataset 4 - Hospital Locations and Capacity (U.S. Homeland Infrastructure Foundation-Level Data)

In [39]:
hosp_req = requests.get('https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.geojson')

In [40]:
hosp_json = hosp_req.json()
hosp_json['features'][0]

hosp_data = [x['properties'] for x in hosp_json['features']]

hosp_df = pd.DataFrame(hosp_data)

hosp_df.head()

,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,TELEPHONE,TYPE,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,1001,0007892363,COLORADO RIVER MEDICAL CENTER,1401 BAILEY AVENUE,NEEDLES,CA,92363,NOT AVAILABLE,NOT AVAILABLE,GENERAL ACUTE CARE,...,2014-02-10T00:00:00.000Z,http://www.cityofneedles.com/Hospitals.asp,NOT AVAILABLE,NOT AVAILABLE,06,GOVERNMENT - LOCAL,-999,25,NOT AVAILABLE,Y
1,1002,0001392120,ALVARADO HOSPITAL MEDICAL CENTER,6655 ALVARADO ROAD,SAN DIEGO,CA,92120,NOT AVAILABLE,NOT AVAILABLE,GENERAL ACUTE CARE,...,2014-02-10T00:00:00.000Z,http://www.alvaradohospital.com,NOT AVAILABLE,NOT AVAILABLE,06,PROPRIETARY,-999,226,NOT AVAILABLE,N
2,1003,0001591942,ALVARADO PARKWAY INSTITUTE B.H.S.,7050 PARKWAY DRIVE,LA MESA,CA,91942,NOT AVAILABLE,NOT AVAILABLE,PSYCHIATRIC,...,2014-02-10T00:00:00.000Z,http://www.apibhs.com,NOT AVAILABLE,NOT AVAILABLE,06,PROPRIETARY,-999,66,NOT AVAILABLE,N
3,1004,0023292104,KINDRED HOSPITAL - SAN DIEGO,1940 EL CAJON BOULEVARD,SAN DIEGO,CA,92104,NOT AVAILABLE,NOT AVAILABLE,GENERAL ACUTE CARE,...,2014-02-10T00:00:00.000Z,http://www.kindredsandiego.com,NOT AVAILABLE,NOT AVAILABLE,06,PROPRIETARY,-999,70,NOT AVAILABLE,N
4,1005,0034392050,PARADISE VALLEY HOSPITAL,2400 EAST FOURTH STREET,NATIONAL CITY,CA,92050,NOT AVAILABLE,NOT AVAILABLE,GENERAL ACUTE CARE,...,2014-02-10T00:00:00.000Z,http://www.paradisevalleyhospital.org,NOT AVAILABLE,NOT AVAILABLE,06,PROPRIETARY,-999,227,NOT AVAILABLE,N
